# Analysis

In [1]:
import pandas as pd

In [2]:
arima_predictions = pd.read_csv('data/arima_predictions', index_col='date')
arima_predictions.index = pd.to_datetime(arima_predictions.index)

prophet_predictions = pd.read_csv('data/prophet_predictions.csv', index_col='date')
prophet_predictions.index = pd.to_datetime(arima_predictions.index)

In [4]:
arima_predictions.tail()

,65807,65802,65804,65810,65806,65809
date,,,,,,
2019-12-01,137481.612848,100931.114780,168107.429120,208566.975000,64758.461240,314575.448793
2020-01-01,138193.703838,101074.440374,169160.295731,208933.789680,64901.576122,316724.333984
2020-02-01,138907.295866,101217.932291,170215.629355,209301.077634,65046.137845,318882.407084
2020-03-01,139622.377104,101361.578804,171273.430893,209668.783798,65192.146407,321049.672687
2020-04-01,140338.942900,101505.374645,172333.700868,210036.867794,65339.601810,323226.133471


In [5]:
prophet_predictions.tail()

,65807,65802,65804,65810,65806,65809
date,,,,,,
2019-12-01,127070.233087,99174.442951,148194.536870,202212.222251,63707.912363,265616.752851
2020-01-01,127517.386168,99590.750794,148669.931055,202847.907779,63653.907614,266730.304083
2020-02-01,127809.488868,99904.390392,149031.206487,203300.825200,63584.750557,267542.519975
2020-03-01,128064.701521,100012.391682,149444.405223,203721.232062,63874.360932,268200.753897
2020-04-01,128651.011816,100255.124506,149841.237435,204064.850838,63873.122266,267778.466056


## RMSE comparison

In [41]:
arima_rmses = pd.read_csv('data/arima_rmses.csv', index_col=0)
arima_rmses.set_index('zip_code', inplace=True)

fb_rmses = pd.read_csv('data/fb_rmses.csv', index_col=0)
fb_rmses.set_index('zip_code', inplace=True)

df_rmse = pd.concat([arima_rmses, fb_rmses], axis=1)

In [42]:
def best_model(row):
    if row['arima_rmse'] < row['fb_rmse']:
        val = 'arima'
    else:
        val = 'fb'
    return val

In [43]:
df_rmse['best_model'] = df_rmse.apply(best_model, axis=1)
df_rmse

,arima_rmse,fb_rmse,best_model
zip_code,,,
65807,1731,3829,arima
65802,3342,3074,fb
65804,2229,4339,arima
65810,4222,6539,arima
65806,3073,3740,arima
65809,11553,13054,arima


The RMSE is lower on all ARIMA models except for zip code 65802.

## Zip Code Selection

In [39]:
from scipy import stats
import datetime as dt

In [44]:
arima_predictions

,65807,65802,65804,65810,65806,65809,date_ordinal
date,,,,,,,
2018-05-01,124546.038914,97684.464161,148184.832451,201396.968517,62314.178082,273671.988982,736815
2018-06-01,125169.802562,98064.249991,149353.760900,202105.993874,62429.803003,276407.338543,736846
2018-07-01,125760.381789,98360.364776,150456.588094,202585.449081,62546.874765,278833.986536,736876
2018-08-01,126364.401331,98595.126886,151522.163437,202962.986466,62665.393367,281084.685736,736907
2018-09-01,126997.888221,98788.004939,152567.198616,203300.201554,62785.358809,283236.752389,736938
2018-10-01,127659.710730,98954.192982,153601.374266,203625.686574,62906.771091,285335.216077,736968
2018-11-01,128341.930177,99104.630583,154630.298121,203951.827314,63029.630212,287406.302377,736999
2018-12-01,129035.991324,99246.667011,155657.218590,204283.265627,63153.936174,289465.285268,737029
2019-01-01,129735.463268,99384.911631,156684.017391,204621.108428,63279.688976,291521.060436,737060


In [45]:
arima_predictions['date_ordinal'] = arima_predictions.index.map(dt.datetime.toordinal)
prophet_predictions['date_ordinal'] = prophet_predictions.index.map(dt.datetime.toordinal)

print('Arima model slopes')
for col in arima_predictions.columns[:-1]:
    slope = stats.linregress(arima_predictions['date_ordinal'], arima_predictions[col])[0]
    print(col, 'slope:', round(slope,2))
    
print('\nProphet model slopes')    
for col in arima_predictions.columns[:-1]:
    slope = stats.linregress(prophet_predictions['date_ordinal'], prophet_predictions[col])[0]
    print(col, 'slope:', round(slope,2))

Arima model slopes
65807 slope: 22.76
65802 slope: 4.99
65804 slope: 34.19
65810 slope: 11.8
65806 slope: 4.32
65809 slope: 69.41

Prophet model slopes
65807 slope: 12.9
65802 slope: 8.21
65804 slope: 11.88
65810 slope: 15.84
65806 slope: 4.36
65809 slope: 12.58


Choosing the correct model for each zip code, we get the following results:

- 65809: 69.41
- 65804: 34.19
- 65807: 22.76
- 65810: 11.8
- 65802: 8.21
- 65806: 4.32

A steep slope indicates a quick rise in value in the future, indicating a good investment. Based on this, zip code 65809 would be a good investment. However, the RMSE values for that zip code's model are much higher than the others, possibly indicating a poorer accuracy forecast or more volatility. Instead, the second steepest slope, zip code 65804, should be chosen as the top investment opportunity.

## Conclusion

Based on the models, we have concluded that zip code 65804 would be the best zip code for the home renovation company to invest their money in. If focusing in the 65804 area, the company should see the average housing prices continuing to increase over the next two years.

## Future Work

It is important to note that these values are already old. Even our 2-year prediction is already 9 months out of date. As well, the chosen ARIMA models had apparent overfitting issues based on their prediction charts, and larger confidence intervals as well. More work would need to be done before making actual investments based on these models.

More recent data would be needed in order to make this model useful for actual investments. If obtained, it could also be used as a sort of holdout dataset, verifying our current conclusions. We have also chosen a 2-year forecast range arbitrarily. In order to improve our model, we would need to perform an analysis on what range these models are most accurate over, and what time frame is useful in the field of housing investments.